# How to Perform a Birefringence Simulation

Consider a ray with wavevector $\mathbf{k}$ incident to an intercept between two media with diectric tensors $\varepsilon_1$ and $\varepsilon$. The point where the incoming ray reaches the intercept is the origin of the global reference frame. By default, all parameters defining incoming ray, intercept, and medium are given in their local form and independent from each other. To transform these parameters to their global form, a rotation needs to be applied. In the given Python script, the rotation is performed with a tensor which is a function of two specific angles:
* $\alpha_0$ - the angle between the global $z$-axis and the local $z$-axis in global $x$-$z$-plane
* $\alpha_1$ - the angle between the local $z$-axis and the global $x$-$z$-plane

The reason to define the rotation-angles in this way is founded in the default traveling direction of the incoming ray, which is along local $z$.
The tensor governing the rotation thus writes as

$$
\text{RotM}(\alpha_0, \alpha_1) = 
\begin{pmatrix}
\cos(\alpha_0) & -\sin(\alpha_0)\cdot\sin(\alpha_1) & \sin(\alpha_0)\cdot\cos(\alpha_1)\\
0 & \cos(\alpha_1) & \sin(\alpha_1)\\
-\sin(\alpha_0) & -\cos(\alpha_0)\cdot\sin(\alpha_1) & \cos(\alpha_1)\cdot\cos(\alpha_0)
\end{pmatrix}.
$$

This kind of rotation is applied to the incoming wavevector, the dielectric tensors of surrounding medium and birefringent medium, and the surface-normal of the intercept, leading to in total 8 rotation angles which need to be defined in advance:
* $\alpha$, $\beta$ - the rotation angles for the $\mathbf{k}$ vector
* $\gamma_1$, $\delta_1$ - the rotation angles for $\varepsilon_1$
* $\delta_2$, $\delta_2$ - the rotation angles for $\varepsilon$
* $\epsilon$, $\zeta$ - the rotation angles for the surface-normal of the intercept $\mathbf{N}$.

Obviously, the rotation for $\varepsilon$ is not independent from the orientation of $\mathbf{N}$. Therefore, the resulting rotation tensors are defined as follows:
* $\text{RotMk} = \text{RotM}(\alpha, \beta)$ - rotation matrix of $\mathbf{k}$
* $\text{RotMN1} = \text{RotM}(\gamma_1, \delta_1)$ - rotation matrix of $\varepsilon_1$
* $\text{RotMN2} = \text{RotM}(\gamma_2 - \epsilon, \delta_2 - \zeta)$ - rotation matrix of $\varepsilon$
* $\text{RotMk} = \text{RotM}(\epsilon, \zeta)$ - rotation matrix of $\mathbf{N}$.

Since this script has been written originally with the application to uniaxial crystals in mind, the considerations until now do not include a rotation of the dieltric tensors around local $z$. Therefore, another rotation tensor is needed for which we use an Euler-transformation by applying the scipy.spatial.transform.Rotation module (denominated as RotMNz).
Thus, the global tensors $\varepsilon_1^\text{gl}$ and $\varepsilon^\text{gl}$ can be determined with

$$
\varepsilon_1^\text{gl} = \text{RotMNz}\cdot\biggl(\text{RotMN1}\cdot\varepsilon_1\cdot\text{RotMN1}^\text{T}\biggr)\cdot\text{RotMNz}^\text{T}
$$
and
$$
\varepsilon^\text{gl} = \text{RotMNz}\cdot\biggl(\text{RotMN2}\cdot\varepsilon\cdot\text{RotMN2}^\text{T}\biggr)\cdot\text{RotMNz}^\text{T},
$$

respectively. The material under consideration is defined via its indicatrix, that is the refractive index ellipsoid which is connected to the dielectric tensor $\varepsilon$ via (neglecting absorption)
$$
\varepsilon = 
\begin{pmatrix}
n_x^2 & 0 & 0\\
0 & n_y^2 & 0\\
0 & 0 & n_z^2
\end{pmatrix}.
$$

With this definition, the dielectric tensor is given in its local reference frame (the sample). By default, $\varepsilon$ is defined so that $n_z$ is along the optical axis.

## Solving for $\mathbf{k}$ and n

With the above inputm, it is now possible to calculate the refraction indexex of all eigenstates of the ray inside the crystal